In [2]:
SEED=185

In [4]:
import pandas as pd
import numpy as np

from runModels import runModels

from sklearn import preprocessing
from sklearn.svm import SVC

In [5]:
DATA='data/mamografias.csv'

# Función para leer los datos
def readData(data_file):
    return pd.read_csv(data_file,sep=',', na_values='?')

data = readData(DATA) # Lectura de los datos

data.rename(columns = {'BI-RADS':'BiRads'}, inplace = True) # Para poder referirnos a esta columna como data.BiRads
data.BiRads.replace(0,pd.NA,inplace=True) # BiRads 0 significa radiografía insuficiente
data.Shape.replace('N',pd.NA,inplace=True) # Lo mismo pasa con Shape N
data.replace(pd.NA, np.nan, inplace=True)

data=data.dropna() # Eliminamos las instancias con valores perdidos
data.shape[0] # Nos quedamos con 825 instancias

# La distribución de la característica densidad es prácticamente degenerada en 3
data.drop('Density', axis='columns',inplace=True)
data.BiRads=data.BiRads.replace(6,5).replace(2,4).replace(3,4)

# Sklearn necesita datos numéricos (aunque sean nominales)
le = preprocessing.LabelEncoder()
data.Shape = le.fit_transform(data.Shape)
data.Severity = le.fit_transform(data.Severity)

dataArray=np.array(data)

# Separamos en datos y target (label)
x=dataArray[:,:-1]
y=dataArray[:,-1]

# Binarización de los atributos nominales
x_nom=x[:,2:4] # Columnas con variables nominales: Shape y Margin
x=np.delete(x,[2,3],axis=1) # Elimino las columnas
onehot = preprocessing.OneHotEncoder(sparse=False)
x_nom = onehot.fit_transform(x_nom)
x=np.hstack((x,x_nom)) # Lo vuelvo a pegar

scaler = preprocessing.StandardScaler()
x=scaler.fit_transform(x)

In [11]:
# C-Support Vector Machine con distintos valores del parámetor C
svc=[SVC(C=c, random_state=SEED) for c in [0.1,0.25,0.5,1,5,10,50]]
model_names=['SVM-C'+str(c) for c in [0.1,0.25,0.5,1,5,10,50]]

In [12]:
runModels(x,y,'SVM_tuning',svc,model_names)